# É provável que oi cliente receba um pagamento de seguro?

# Contents <a id='back'></a>

* [Introdução](#intro)
* [Etapa 1. Visão geral dos dados](#data_review)
* [Etapa 2. Modelo](#model)
* [Etapa 3. Ofuscar Dados](#obfuscate)
* [Conclusões](#end)

A companhia de seguros Proteja Seu Amanhã quer resolver algumas tarefas com a ajuda de aprendizado de máquina e você precisa avaliar a possibilidade de fazê-lo.

- Tarefa 1: Encontrar clientes semelhantes a um determinado cliente. Isso vai ajudar os agentes da empresa com tarefas de marketing.
- Tarefa 2: Predizer se um novo cliente provavelmente receberá um pagamento de seguro. Um modelo de predição pode ser melhor do que um modelo dummy?
- Tarefa 3: Predizer o número de pagamentos de seguro que um novo cliente provavelmente receberá usando um modelo de regressão linear.
- Tarefa 4: Proteger os dados pessoais dos clientes sem estragar o modelo da tarefa anterior. É necessário desenvolver um algoritmo de transformação de dados que tornaria difícil recuperar informações pessoais se os dados caíssem nas mãos erradas. Isso é chamado de mascaramento de dados ou ofuscação de dados. Mas os dados devem ser protegidos de forma que a qualidade dos modelos de aprendizado de máquina não piore. Você não precisa escolher o melhor modelo, só prove que o algoritmo funciona corretamente.

# Pré-processamento de dados & Exploração

## Inicialização

In [1]:
pip install scikit-learn --upgrade

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/17/1c/ccdd103cfcc9435a18819856fbbe0c20b8fa60bfc3343580de4be13f0668/scikit_learn-1.5.2-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for threadpoolctl>=3.1.0 from https://files.pythonhosted.org/packages/4b/2c/ffbf7a134b9ab11a67b0cf0726453cedd9c5043a4fe7a35d1cefa9a1bcfb/threadpoolctl-3.5.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
    --------------------------------------- 0.2/11.0 MB 4.1 MB/s eta 0:00:03
   -- ------------------------------------- 0.7/11.0 MB 7.3 MB/s eta 0:00:02
   ---- ----------------------------------- 1.2/11.0 MB 8.7 MB/s eta 0:00:02
   ------ --------------------------------- 1.9/11.0 MB 10.0 MB/s eta 0:00:01
   --------- ------------------------------ 2.7/11.0 MB 11.3 MB/s eta 0:00:01
   ----------- --------------

## Etapa 1. Visão geral dos dados <a id='data_review'></a>

In [2]:
# Importando as bibliotecas necessárias
import numpy as np  # Biblioteca para operações matemáticas e manipulação de arrays
import pandas as pd  # Biblioteca para manipulação e análise de dados em formato tabular

import seaborn as sns  # Biblioteca para visualização de dados baseada no Matplotlib

import sklearn.linear_model  # Módulo do scikit-learn para modelos de regressão linear
import sklearn.metrics  # Módulo do scikit-learn para avaliação de modelos
import sklearn.neighbors  # Módulo do scikit-learn para algoritmos de vizinhos mais próximos
import sklearn.preprocessing  # Módulo do scikit-learn para pré-processamento de dados

from sklearn.model_selection import train_test_split  # Função para dividir os dados em conjuntos de treino e teste

from IPython.display import display  # Função para exibir objetos em um formato mais elegante no Jupyter Notebook

## Carregar Dados

Carregue os dados e faça uma verificação básica de que estão livres de problemas óbvios.

In [3]:
df = pd.read_csv('/datasets/insurance_us.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/insurance_us.csv'

Renomeamos as colunas para tornar o código mais consistente com seu estilo.

In [ ]:
df = df.rename(columns={'Gender': 'gender', 'Age': 'age', 'Salary': 'income', 'Family members': 'family_members', 'Insurance benefits': 'insurance_benefits'})

In [ ]:
df.sample(10)

In [ ]:
df.info()

In [ ]:
# podemos querer corrigir o tipo de idade (de float para int), embora isso não seja crítico
df['age'] = df['age'].astype('int')
# escreva sua conversão aqui se você escolher:

In [ ]:
# verifique se a conversão foi bem-sucedida

In [ ]:
# agora dê uma olhada nas estatísticas descritivas dos dados.
# Parece que está tudo bem?

In [ ]:
df.sample(10)

In [ ]:
df.info()

In [ ]:
df.describe()

## AED

Vamos verificar rapidamente se existem determinados grupos de clientes observando o gráfico de pares.

In [ ]:
g = sns.pairplot(df, kind='hist')
g.fig.set_size_inches(12, 12)

Ok, é um pouco difícil identificar grupos óbvios (clusters), pois é difícil combinar várias variáveis simultaneamente (para analisar distribuições multivariadas). É aí que Álgebra Linear e Aprendizado de Máquina podem ser bastante úteis.

# Tarefa 1. Clientes Similares

Na linguagem de AM, é necessário desenvolver um procedimento que retorne k vizinhos mais próximos (objetos) para um determinado objeto com base na distância entre os objetos.
Você pode querer rever as seguintes lições (capítulo -> lição)- Distância Entre Vetores -> Distância Euclidiana
- Distância Entre Vetores -> Distância de Manhattan

Para resolver a tarefa, podemos tentar diferentes métricas de distância.

Escreva uma função que retorne k vizinhos mais próximos para um n-ésimo objeto com base em uma métrica de distância especificada. O número de pagamentos de seguro recebidos não deve ser levado em consideração para esta tarefa. 

Você pode usar uma implementação pronta do algoritmo kNN do scikit-learn (verifique [o link](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html#sklearn.neighbors.NearestNeighbors)) ou usar a sua própria.
Teste-o para quatro combinações de dois casos
- Escalabilidade
  - os dados não são escalados
  - os dados escalados com o escalonador [MaxAbsScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MaxAbsScaler.html) 
- Métricas de distância
  - Euclidiana
  - Manhattan

Responda às perguntas:
- Os dados não escalados afetam o algoritmo kNN? Se sim, como isso acontece?
-Quão semelhantes são os resultados usando a métrica de distância de Manhattan (independentemente da escalabilidade)?

In [ ]:
feature_names = ['gender', 'age', 'income', 'family_members']

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
def get_knn(df, n, k, metric):
    
    """
    Retorna os vizinhos mais próximos de k

    :param df: DataFrame pandas usado para encontrar objetos semelhantes dentro de    
    :param n: número do objeto pelo qual os vizinhos mais próximos são procurados
    :param k: o número dos vizinhos mais próximos a serem retornados
    :param metric: nome da métrica de distância    """

    nbrs = NearestNeighbors(n_neighbors=k,metric=metric).fit(df[feature_names])
    nbrs_distances, nbrs_indices = nbrs.kneighbors([df.iloc[n][feature_names]], k, return_distance=True)
    
    df_res = pd.concat([
        df.iloc[nbrs_indices[0]], 
        pd.DataFrame(nbrs_distances.T, index=nbrs_indices[0], columns=['distance'])
        ], axis=1)
    
    return df_res

Escalando os dados

In [ ]:
feature_names = ['gender', 'age', 'income', 'family_members']

transformer_mas = sklearn.preprocessing.MaxAbsScaler().fit(df[feature_names].to_numpy())

df_scaled = df.copy()
df_scaled.loc[:, feature_names] = transformer_mas.transform(df[feature_names].to_numpy())

In [ ]:
df_scaled.sample(5)

Agora, vamos obter registros semelhantes para um determinado registro para cada combinação

In [ ]:
get_knn(df, 1, 5, "manhattan")

In [ ]:
get_knn(df_scaled, 1, 5, "manhattan")

In [ ]:
get_knn(df, 1, 5, "euclidean")

In [ ]:
get_knn(df_scaled, 1, 5, "euclidean")

Respostas para as perguntas

**Os dados não escalados afetam o algoritmo kNN? Se sim, como isso acontece?** 

Sim, afetam a imprescindibilidade, podendo ocasionar em erros de predições.

**Quão semelhantes são os resultados usando a métrica de distância de Manhattan (independentemente da escalabilidade)?** 
É menos preciso das demais, gerando grande distancia entre os dados.

# Tarefa 2. É provável que o cliente receba um pagamento do seguro?

Em termos de aprendizado de máquina, podemos olhar para isso como uma tarefa de classificação binária.

Com os pagamentos de seguro sendo mais do que zero como objetivo, avalie se a abordagem da classificação kNN pode ser melhor do que um modelo dummy.

Instruções:
- Construa um classificador baseado em kNN e meça sua qualidade com a métrica F1 para k=1..10 tanto para os dados originais quanto para os escalados. Seria interessante ver como k pode influenciar a métrica de avaliação e se a escalabilidade dos dados faz alguma diferença. Você pode usar uma implementação pronta do algoritmo de classificação kNN do scikit-learn (verifique [o link](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)) ou usar a sua própria.
- Construa o modelo dummy, que é aleatório para este caso. Deve retornar com alguma probabilidade o valor "1". LVamos testar o modelo com quatro valores de probabilidade: 0, a probabilidade de fazer qualquer pagamento de seguro, 0,5, 1.

A probabilidade de fazer qualquer pagamento de seguro pode ser definida como

$$
P\{\text{pagamento de seguro recebido}= número de clientes que receberam qualquer pagamento de seguro}}{\text{número total de clientes}}.
$$

Divida os dados inteiros na proporção 70:30 para as partes de treinamento/teste.

In [ ]:
# calcule a meta
df['insurance_benefits_received'] = df['insurance_benefits'].sum()/df['insurance_benefits'].count()

In [ ]:
# verifique o desequilíbrio de classe com value_counts()
df['insurance_benefits']


In [ ]:
#substituindo valores maiores de 1 em 1
df['insurance_benefits'] = df['insurance_benefits'].mask(df['insurance_benefits'] > 1, 1)
df_scaled['insurance_benefits'] = df_scaled['insurance_benefits'].mask(df_scaled['insurance_benefits'] > 1, 1)

In [ ]:
df['insurance_benefits'].value_counts(normalize=True)

In [ ]:
df_scaled['insurance_benefits'].value_counts(normalize=True)

In [ ]:
df['insurance_benefits_received'] = df['insurance_benefits'].sum()/df['insurance_benefits'].count()

In [ ]:
df_scaled['insurance_benefits_received'] = df_scaled['insurance_benefits'].sum()/df_scaled['insurance_benefits'].count()

In [ ]:
df

In [ ]:
df_scaled

In [ ]:
def eval_classifier(y_true, y_pred):
    
    f1_score = sklearn.metrics.f1_score(y_true, y_pred)
    print(f'F1: {f1_score:.2f}')
    
# se você tiver um problema com a linha a seguir, reinicie o kernel e execute o caderno novamente
    cm = sklearn.metrics.confusion_matrix(y_true, y_pred, normalize='all')
    print('Matriz de Confusão')
    print(cm)

In [ ]:
df['insurance_benefits']

In [ ]:
# gerando saída de um modelo aleatório

def rnd_model_predict(P, size, seed=42):

    rng = np.random.default_rng(seed=seed)
    return rng.binomial(n=1, p=P, size=size)

In [ ]:
for P in [0, df['insurance_benefits'].sum() / len(df), 0.5, 1]:

    print(f'A probabilidade: {P:.2f}')
    y_pred_rnd = rnd_model_predict(P,5000)
        
    eval_classifier(df['insurance_benefits'], y_pred_rnd)
    
    print()

In [ ]:
for P in [0, df_scaled['insurance_benefits'].sum() / len(df), 0.5, 1]:

    print(f'A probabilidade: {P:.2f}')
    y_pred_rnd = rnd_model_predict(P,5000)
        
    eval_classifier(df_scaled['insurance_benefits'], y_pred_rnd)
    
    print()

Dos dados escalados e não escalados podemos ver um resultado idêntico. O resultado mais plausível seria o de 50% de probabilidade sendo seu F1 de 0.20 com a matrix de confusão melhor distribuida.

# Etapa 2. Modelo <a id='model'></a>

# Tarefa 3. Regressão (com Regressão Linear)

Com os pagamentos de seguro como objetivo, avalie qual seria o REQM para um modelo de Regressão Linear.

Construa sua própria implementação de Regressão Linear. Para isso, lembre-se de como a solução da tarefa de regressão linear é formulada em termos de Álgebra linear. Verifique o REQM para os dados originais e os escalados. Você pode ver alguma diferença no REQM entre esses dois casos?

Vamos denotar
- $X$ — matriz de características, cada linha é um caso, cada coluna é uma característica, a primeira coluna consiste em unidades
- $y$ — objetivo (um vetor)
- $\hat{y}$ — objetivo estimado (um vetor)- $w$ — vetor de peso

A tarefa de regressão linear na linguagem de matrizes pode ser formulada como
$$
y = Xw
$$

O objetivo do treinamento, então, é encontrar os $w$ que minimizaria a distância L2 (EQM) entre $Xw$ e $y$:

$$
\min_w d_2(Xw, y) \quad \text{or} \quad \min_w \text{MSE}(Xw, y)
$$

Parece que há uma solução analítica para a questão acima:

$$
w = (X^T X)^{-1} X^T y
$$

A fórmula acima pode ser usada para encontrar os pesos $w$ e o último pode ser usado para calcular valores preditos

$$
\hat{y} = X_{val}w
$$

Dividi todos os dados na proporção 70:30 para as partes de treinamento/validação. Usei a métrica REQM para a avaliação do modelo.

In [ ]:
class MyLinearRegression:
    
    def __init__(self):
        
        self.weights = None
    
    def fit(self, X, y):
        
        # somando as unidades
        X2 = np.append(np.ones([len(X), 1]), X, axis=1)
        self.weights = np.linalg.inv(X2.T.dot(X2)).dot(X2.T).dot(y)

    def predict(self, X):
        
        # somando as unidades
        X2 = np.append(np.ones([len(X), 1]), X, axis=1).dot(self.weights)
        return X2

In [ ]:
def eval_regressor(y_true, y_pred):
    
    rmse = math.sqrt(sklearn.metrics.mean_squared_error(y_true, y_pred))
    print(f'REQM: {rmse:.2f}')
    
    r2_score = math.sqrt(sklearn.metrics.r2_score(y_true, y_pred))
    print(f'R2: {r2_score:.2f}')    

In [ ]:
X = df[['age', 'gender', 'income', 'family_members']].to_numpy()
y = df['insurance_benefits'].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=12345)

lr = MyLinearRegression()

lr.fit(X_train, y_train)
print(lr.weights)



In [ ]:
lr.fit(X_test, y_test)
print(lr.weights)

In [ ]:
import math

In [ ]:
y_test_pred = lr.predict(X_test)
eval_regressor(y_test, y_test_pred)

Criamos um modelo de regressão Linear com REQM de 0.23 e um R2 de 0.66.

# Tarefa 4. Ofuscando dados<a id='obfuscate'></a>

In [ ]:
personal_info_column_list = ['gender', 'age', 'income', 'family_members']
df_pn = df[personal_info_column_list]

In [ ]:
df_pn

In [ ]:
X = df_pn.to_numpy()

In [ ]:
X

Gerando uma matriz $P$ aleatória.

In [ ]:
rng = np.random.default_rng(seed=42)
P = rng.random(size=(X.shape[1], X.shape[1]))

In [ ]:
P

Verificando se a matriz $P$ é invertível

In [ ]:
Pi = np.linalg.inv(P)
Pi

In [ ]:
Pi @ P

Você consegue adivinhar a idade ou a renda dos clientes após a transformação?

Não, pois os dados estão ofuscados.

In [ ]:
X3 = X.dot(P)

print(X3) 

### Recupendando os dados ofuscados

In [ ]:
Xof = (X3.dot(Pi))
Xof


Imprimindo todos os três casos para alguns clientes- Os dados originais
- O transformado
- O invertido (recuperado)

In [ ]:
#Os dados originais
X

In [ ]:
#O transformado
X3

In [ ]:
#O invertido (recuperado)
Xof

Os valores de 0 foram os mais afetados, dado o ofuscamento esses valores quando são multiplicados pela matrix invertível 𝑃. 

## Teste de regressão linear com ofuscação de dados

Agora, vamos provar que a Regressão Linear pode funcionar computacionalmente com a transformação de ofuscação escolhida.
Crie um procedimento ou uma classe que execute a Regressão Linear opcionalmente com a ofuscação. Você pode usar uma implementação pronta de Regressão Linear do scikit-learn ou sua própria.

Execute a Regressão Linear para os dados originais e os ofuscados, compare os valores previstos e os valores da métrica $R^2$ do REQM. Há alguma diferença?

**Procedimento**

- Crie uma matriz quadrada $P$ de números aleatórios.
- Verifique se é invertível. Caso contrário, repita o primeiro ponto até obtermos uma matriz invertível.
- Use $XP$ como a nova matriz de características

In [ ]:
rngtest = np.random.default_rng(seed=36)
P = rngtest.random(size=(X.shape[1], X.shape[1]))

In [ ]:
P

In [ ]:
Pi = np.linalg.inv(P)
Pi

In [ ]:
Pi @ P

In [ ]:
class MyLinearRegressionOvershadowed:
    
    def __init__(self):
        
        self.weights = None
    
    def fitovershadowed(self, X, y):
        
        # somando as unidades
        X2 = np.append(np.ones([len(X), 1]), X, axis=1)
        self.weights = np.linalg.inv((X2@P).T.dot(X2@P)).dot((X2@P).T).dot(y)

    def predictovershadowed(self, X):
        
        # somando as unidades
        X2 = np.append(np.ones([len(X), 1]), X, axis=1).dot(self.weights)
        return X2

In [ ]:
lrOvershadowed = MyLinearRegression()

lrOvershadowed.fit(X_train, y_train)
print(lrOvershadowed.weights)


In [ ]:
lrOvershadowed.fit(X_test, y_test)
print(lr.weights)

In [ ]:
y_test_predlrOvershadowed = lrOvershadowed.predict(X_test)
eval_regressor(y_test, y_test_predlrOvershadowed)

Obtivemos os mesmo resultados do que os dados não ofuscados, quer dizer que nosso trabalho foi concluído corretamente.

## Conclusão geral <a id='end'></a>

Neste projeto, importamos bibliotecas como "pandas" e "numpy" que habitualmente já utilizamos, sklearn para criação do nosso modelo, "Seaborn" para representação em gráficos.

Realizamos o pre-processamento, para que não tivesse acontecido algum problema na criação do nosso modelo.

Usamos o algoritimo KNN para procurar clientes similares, com auxilio das métricas Distância Euclidiana e Distância Manhattan, trabalhamos com os dados escalados e não escalados para ter melhor visualização e precisão em nossa predição. Desenvolvemos um protótipo de um modelo de aprendizado de máquina para saber se é provável que o cliente receba um pagamento do seguro, com a probalidade de 50% e um F1: 0.20 como a melhor alternativa, aplicamos uma regressão Linear com REQM: 0.23 R2: 0.66 com os dados ofuscados e não ofuscados.